In [25]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pprint

In [26]:
file = "turkiye_posta_kodlari.xlsx"
df = pd.read_excel(file)
df.head()

,il,ilçe,semt_bucak_belde,Mahalle,PK
0,ADANA,ALADAĞ,ALADAĞ,AKPINAR MAH,1720
1,ADANA,ALADAĞ,ALADAĞ,BAŞPINAR MAH,1720
2,ADANA,ALADAĞ,ALADAĞ,MANSURLU MAH,1720
3,ADANA,ALADAĞ,ALADAĞ,SİNANPAŞA MAH,1720
4,ADANA,ALADAĞ,MADENLİ,AKÖREN MAH,1722


In [29]:
city_col, district_col, neigh_col = "il", "ilçe", "Mahalle"

# Filtrelenecek iller
target_cities = {"AYDIN", "MUĞLA", "DENİZLİ", "İZMİR", "MANİSA"}

# Sadece gerekli sütunlar + temel temizlik
tmp = df[[city_col, district_col, neigh_col]].copy()
for c in (city_col, district_col, neigh_col):
    tmp[c] = tmp[c].astype(str).str.strip()

# İlgili illere filtrele
tmp = tmp[tmp[city_col].isin(target_cities)]

# Aynı (il, ilçe, mahalle) tekrarlarını kaldır
unique_rows = tmp.drop_duplicates(subset=[city_col, district_col, neigh_col])

# Mahallelerin hangi illerde göründüğünü bul
city_groups = (
    unique_rows.groupby(neigh_col)[city_col]
    .nunique()
    .reset_index(name="city_count")
)

# Çakışmalı mahalle adları: birden fazla farklı ilde görünenler
conflicting_names = city_groups.loc[city_groups["city_count"] > 1, neigh_col]

# Çakışmaları detaylı listele
conflicts = (
    unique_rows[unique_rows[neigh_col].isin(conflicting_names)]
    .sort_values([neigh_col, city_col, district_col])
    .reset_index(drop=True)
)

# Çıktı
print("ÇAKIŞMALAR (mahalle adı aynı, ve AYDIN-MUĞLA-DENİZLİ-İZMİR-MANİSA illerinden en az 2'sinde geçiyor):")
current = None
for _, r in conflicts.iterrows():
    name = r[neigh_col]
    if name != current:
        print(f"\n--- ÇAKIŞMA: {name} ---")
        current = name
    print(f"{r[city_col]} | {r[district_col]} | {r[neigh_col]}")

ÇAKIŞMALAR (mahalle adı aynı, ve AYDIN-MUĞLA-DENİZLİ-İZMİR-MANİSA illerinden en az 2'sinde geçiyor):

--- ÇAKIŞMA: 4 EYLÜL MAH ---
MANİSA | KULA | 4 EYLÜL MAH
İZMİR | TİRE | 4 EYLÜL MAH

--- ÇAKIŞMA: 75. YIL MAH ---
MANİSA | YUNUSEMRE | 75. YIL MAH
İZMİR | BAYRAKLI | 75. YIL MAH

--- ÇAKIŞMA: ACARLAR MAH ---
AYDIN | İNCİRLİOVA | ACARLAR MAH
MANİSA | TURGUTLU | ACARLAR MAH
İZMİR | SELÇUK | ACARLAR MAH

--- ÇAKIŞMA: ADALET MAH ---
DENİZLİ | MERKEZEFENDİ | ADALET MAH
İZMİR | BAYRAKLI | ADALET MAH

--- ÇAKIŞMA: ADNAN MENDERES MAH ---
AYDIN | EFELER | ADNAN MENDERES MAH
AYDIN | KOÇARLI | ADNAN MENDERES MAH
MANİSA | GÖRDES | ADNAN MENDERES MAH
MANİSA | ŞEHZADELER | ADNAN MENDERES MAH
İZMİR | TİRE | ADNAN MENDERES MAH

--- ÇAKIŞMA: AHATLAR MAH ---
AYDIN | KÖŞK | AHATLAR MAH
MANİSA | DEMİRCİ | AHATLAR MAH

--- ÇAKIŞMA: AHMETLER MAH ---
MANİSA | DEMİRCİ | AHMETLER MAH
İZMİR | KİRAZ | AHMETLER MAH

--- ÇAKIŞMA: AHMETLİ MAH ---
DENİZLİ | SARAYKÖY | AHMETLİ MAH
MANİSA | KULA | AHMETLİ MAH
İZMİR | 

In [21]:
city_col, district_col, neigh_col = 0, 1, 3

# Recursive defaultdict for arbitrary depth
def tree():
    return defaultdict(tree)

data = tree()

for _, row in df.iterrows():
    city = str(row[city_col]).strip()
    district = str(row[district_col]).strip()
    neigh = str(row[neigh_col]).strip()

    # Ensure city → district → neighbourhood exists
    _ = data[city][district][neigh]

# Convert defaultdicts to plain dicts for readability / export
def to_dict(d):
    if isinstance(d, defaultdict):
        return {k: to_dict(v) for k, v in d.items()}
    return d

nested_dict = to_dict(data)

pp = pprint.PrettyPrinter(indent=2, width=80, sort_dicts=False)
adana = nested_dict["ADANA"]

ilce_sayisi = len(adana)  # ADANA'daki ilçe sayısı
mahalle_sayisi = sum(len(mahalleler) for mahalleler in adana.values())

print("ADANA'daki toplam ilçe sayısı:", ilce_sayisi)
print("ADANA'daki toplam mahalle sayısı:", mahalle_sayisi)

/var/folders/pr/5pprkxc119jbjdpsdpnpr_pm0000gn/T/ipykernel_73856/3630329353.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city = str(row[city_col]).strip()
/var/folders/pr/5pprkxc119jbjdpsdpnpr_pm0000gn/T/ipykernel_73856/3630329353.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  district = str(row[district_col]).strip()
/var/folders/pr/5pprkxc119jbjdpsdpnpr_pm0000gn/T/ipykernel_73856/3630329353.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posit

{ 'ALADAĞ': { 'AKPINAR MAH': {},
              'BAŞPINAR MAH': {},
              'MANSURLU MAH': {},
              'SİNANPAŞA MAH': {},
              'AKÖREN MAH': {},
              'BOZTAHTA MAH': {},
              'BÜYÜKSOFULU MAH': {},
              'CERİTLER MAH': {},
              'DAİLER MAH': {},
              'DARILIK MAH': {},
              'DÖLEKLİ MAH': {},
              'EBRİŞİM MAH': {},
              'EĞNER MAH': {},
              'GERDİBİ MAH': {},
              'GİREĞİYENİKÖY MAH': {},
              'GÖKÇEKÖY MAH': {},
              'KABASAKAL MAH': {},
              'KARAHAN MAH': {},
              'KICAK MAH': {},
              'KIŞLAK MAH': {},
              'KIZILDAM MAH': {},
              'KÖKEZ MAH': {},
              'KÖPRÜCÜK MAH': {},
              'KÜP MAH': {},
              'MADENLİ MAH': {},
              'MAZILIK MAH': {},
              'POSYAĞBASAN MAH': {},
              'TOPALLI MAH': {},
              'UZUNKUYU MAH': {},
              'YETİMLİ MAH': {